In [1]:
import logging
import torch

from envs._2048.vecttrainer import _2048Trainer, init_2048_trainer_from_checkpoint

logging.basicConfig(filename='benchmark.log', filemode='a', level=logging.INFO, format='%(asctime)s %(message)s')
logging.info('Starting benchmark')

In [2]:
NUM_PARALLEL_ENVS = 1
CHECKPOINTS = [
    './checkpoints/0.pt',
    './checkpoints/1-2.pt',
    './checkpoints/2-2.pt',
    './checkpoints/3-2.pt',
    './checkpoints/4-2.pt',
    './checkpoints/5.pt',
]
ITER_AMNTS = [5, 10, 50, 100]
ITER_DEPTHS = [1, 2, 3, 4, 5]

NUM_PARALLEL_ENVS = 10
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
results = dict()
for checkpoint in CHECKPOINTS:
    for iter_amnt in ITER_AMNTS:
        for iter_depth in ITER_DEPTHS:
            logging.info(f'Benchmarking checkpoint {checkpoint} with {iter_amnt} iterations and {iter_depth} depth')
            trainer: _2048Trainer = init_2048_trainer_from_checkpoint(NUM_PARALLEL_ENVS, checkpoint, device)
            trainer.hypers.num_iters_eval = iter_amnt
            trainer.hypers.iter_depth_test = iter_depth
            trainer.run_test_batch()
            results[(checkpoint, iter_amnt, iter_depth)] = trainer.history.eval_metrics